In [ ]:
!pip install pandas bokeh

In [ ]:
import pandas as pd
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category10
from bokeh.plotting import figure, output_notebook, show
from bokeh.io import push_notebook
import ipywidgets as widgets
from IPython.display import display

data = pd.read_csv('../data/cosim/maestro-output.csv')
source = ColumnDataSource(data)

x_column = data.columns[0]
y_columns = list(data.columns)[1:]

p = figure(
    title="Co-simulation outputs",
    width=800,
    height=400,
    x_axis_label=x_column,
    y_axis_label=", ".join(y_columns),
)

lines = {}
line_colors = Category10[len(y_columns)]

for i, y_column in enumerate(y_columns):
    line = p.line(x=x_column, y=y_column, source=source, line_width=2, line_color=line_colors[i], legend_label=y_column)
    lines[y_column] = line

output_notebook()

p.yaxis[0].axis_label = "Multi-model connections"

y_select = widgets.Dropdown(
    options=["All systems"] + y_columns,
    description="Y-axis:"
)

def update_plot(change):
    y_col = y_select.value
    if y_col == "All systems":
        y_columns_to_plot = y_columns
        p.yaxis[0].axis_label = "Multi-model connections"
    else:
        y_columns_to_plot = [y_col]
        p.yaxis[0].axis_label = ", ".join(y_columns_to_plot)

    for column in y_columns:
        lines[column].visible = (column in y_columns_to_plot)

    legend_items = [(y_column, [lines[y_column]]) for y_column in y_columns_to_plot]
    p.legend.items = legend_items

    push_notebook()

y_select.observe(update_plot, names='value')

display(widgets.VBox([y_select]))
show(p, notebook_handle=True)
